In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.svm import SVC
import pickle
from sklearn.compose import make_column_transformer

In [3]:
def clean_data(file_name):
      df = pd.read_csv(file_name)

      # drop all NA value
      col = [col for col in df.columns]
      df.dropna(subset=col, inplace=True)

      #imputing data for numerical value


      # replace the string inside salary into 'B' and 'M"
      df["salary"] = df["salary"].replace(['Less than 1K','1K to 2K','2K to 3K','3K to 4K','4K to 5K'],'B')
      df["salary"] = df["salary"].replace(['5K to 6K','7K to 8K','8K to 9K','9K to 10K','10K or more'],"M")

      # replace string inside 'person_living_in_house' feature
      df["person_living_in_house"] = df["person_living_in_house"].replace(['10 or more'],'10')

      try:
            # remove space in numeric feature. ie: - 5 3 
            mask = df['kids_spending'].str.fullmatch(r'[\d ]*')
            df.loc[:, 'kids_spending'] = df.loc[mask, 'kids_spending'].str.replace(' ', '')

            mask1 = df['house_utility'].str.fullmatch(r'[\d ]*')
            df.loc[:, 'house_utility'] = df.loc[mask1, 'house_utility'].str.replace(' ', '')
      except:
            print('No space need to remove')


      return df

def column_transformer():
      ohe = OneHotEncoder()

      column_trans = make_column_transformer(
       (ohe, ['race', 'gender', 'employment', 'education', 'married',
       'house_type', 'house_value','vehicle', 'transport_use']), remainder='passthrough')

      return column_trans

def execute_SVM(file_to_be_train, predicted_file):
      '''
      Create  model using surveyA.csv
      '''
      # for train dataset
      df_clean = clean_data(file_to_be_train)
      X = df_clean.drop('salary', axis=1)
      
      ohe = OneHotEncoder()

      X_ohe = ohe.fit_transform(X)
      y = df_clean['salary']

      X_train, X_test, y_train, y_test = train_test_split(X_ohe, y, test_size=0.2, random_state=42) 

      # create SVM Model
      svm = SVC(random_state=42, C=1, gamma='scale')
      svm.fit(X_train, y_train)

      # ---------------------------------------------------------------

      # for new dataset to be predicted
      df_clean_pred = clean_data(predicted_file)
      X_from_pred = df_clean_pred.drop('salary', axis=1)
      
      X_ohe_pred = ohe.transform(X_from_pred) # error here 
   
      
      y_pred = svm.predict(X_ohe_pred)
      return y_pred


# function use to execute the model


execute_SVM("surveyA.csv", "surveyACopy.csv")


# Notes:
# theres a problem if the dtype of the new dataset to be predicted
# is different with the trained dataset.
# otherwise, it couldn't be problem.

array(['B', 'M', 'B', 'B', 'B', 'M', 'B', 'M', 'B', 'B', 'B', 'B', 'M',
       'B', 'B', 'B', 'B', 'B', 'B', 'M', 'B', 'B', 'M', 'B', 'B', 'M',
       'B', 'B', 'B', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'M', 'B', 'B',
       'B', 'B', 'M', 'B', 'M', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'M', 'B', 'B', 'B', 'M', 'B', 'M', 'B', 'B', 'B',
       'M', 'B', 'B', 'M', 'B', 'M', 'B', 'B', 'B', 'B', 'M', 'M', 'B',
       'B', 'B', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'M', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'M', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'M', 'B', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'M',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'M', 'B', 'B',
       'M', 'B', 'B', 'M', 'B', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'M', 'B', 'B', 'B', 'M', 'B', 'B',
       'B', 'M', 'B', 'M', 'B', 'M', 'B', 'B', 'M', 'B', 'B', 'B', 'B',
       'B', 'B', 'M', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'M

In [3]:
# generate pickle model 

pickle.dump(SVC, open("INVOKE_ML_Ammar.pkl", 'wb'))